# Classification 

In [ ]:
] add RDatasets

In [ ]:
] add MLBase

In [1]:
using RDatasets
using MLBase

In [2]:
iris = dataset("datasets", "iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [3]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [5]:
irislabelsmap = labelmap(irislabels)

LabelMap (with 3 labels):
[1] setosa
[2] versicolor
[3] virginica


In [6]:
y = labelencode(irislabelsmap, irislabels)  # convert this to numeric counterparts 

150-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [7]:
using Random

In [8]:
function perclass_splits(y, p)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, p) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [9]:
train_ids = perclass_splits(y, 0.7)

103-element Array{Any,1}:
   2
   3
   4
   5
   6
   7
  11
  12
  13
  14
  15
  16
  17
   ⋮
 134
 135
 136
 137
 138
 141
 142
 143
 145
 146
 148
 150

In [10]:
test_ids = setdiff(1:length(y), train_ids)

47-element Array{Int64,1}:
   1
   8
   9
  10
  19
  22
  24
  26
  27
  28
  39
  43
  47
   ⋮
 110
 114
 117
 125
 131
 132
 133
 139
 140
 144
 147
 149

In [25]:
@show size(train_ids)
@show size(test_ids);

size(train_ids) = (103,)
size(test_ids) = (47,)


In [16]:
function assign_class(predictedvalue)
    return argmin(abs.(predictedvalue .- [1, 2, 3]))      # return whichever value the predicted value is closest to 
end 

assign_class (generic function with 1 method)

In [15]:
# to clarify for predicted value of 3.1: 
argmin([2.1, 1.1, 0.9])   

3

In [21]:
assign_class(2.2)

2

## Using Classifiers 

## Decision Trees 

In [ ]:
] add DecisionTree

In [22]:
using DecisionTree

In [23]:
model = DecisionTreeClassifier(max_depth=2)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  nothing
root:                     nothing

In [26]:
DecisionTree.fit!(model, X[train_ids,:], y[train_ids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [28]:
q = X[test_ids,:];
predictions_DT = DecisionTree.predict(model, q)

47-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [29]:
function findaccuracy(predictedvals, groundtruthvals)
    sum(predictedvals .== groundtruthvals) / length(groundtruthvals)
end 

findaccuracy (generic function with 1 method)

In [31]:
findaccuracy(predictions_DT,y[test_ids])

0.9361702127659575

### Random Forests 

In [33]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[train_ids,:], y[train_ids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 6.7
Avg Depth:  5.15

In [35]:
q = X[test_ids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[test_ids])

0.9361702127659575

### Support Vector Machines 

In [ ]:
] add LIBSVM 

In [36]:
using LIBSVM

In [37]:
Xtrain = X[train_ids,:]
ytrain = y[train_ids]

103-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [38]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Int64,Float64}(35, Int32[5, 15, 15], [1, 1, 1, 1, 1, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.3 5.7 … 6.5 5.9; 3.0 4.4 … 3.0 3.0; 1.1 1.5 … 5.2 5.1; 0.1 0.4 … 2.0 1.8], Int32[10, 12, 18, 31, 33, 36, 38, 39, 40, 43  …  86, 88, 89, 91, 92, 93, 98, 99, 102, 103], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 4.8), LIBSVM.SVMNode(1, 4.5), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 7.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 4.9), LIBSVM.SVMNode(1, 5.0)  …  LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.0 0.043651571744575526; 0.6315183672229074 0.9001291780053436; … ; -0.0 -1.0; -0.0 -1.

In [39]:
predictions_SVM, decision_values = svmpredict(model, X[test_ids,:]')
findaccuracy(predictions_SVM, y[test_ids])

0.9574468085106383